In [ ]:
!pip install selenium

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import re
from urllib.parse import urljoin, urlparse
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def initialize_webdriver():
    """웹드라이버 초기화 설정"""
    chrome_settings = Options()
    chrome_settings.add_argument('--headless')
    chrome_settings.add_argument('--no-sandbox')
    chrome_settings.add_argument('--disable-dev-shm-usage')
    chrome_settings.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
    
    browser = webdriver.Chrome(options=chrome_settings)
    return browser

def fetch_webtoon_images(webtoon_title, chapter_url):
    """
    웹툰 챕터의 이미지를 다운로드하는 함수 (Selenium 사용)
    """
    browser = None
    try:
        # 브라우저 초기화
        browser = initialize_webdriver()
        
        # 페이지 접속
        browser.get(chapter_url)
        
        # 페이지 로딩 대기
        WebDriverWait(browser, 15).until(
            EC.presence_of_element_located((By.CLASS_NAME, "wt_viewer"))
        )
        # 챕터 번호 추출
        chapter_num = re.search(r'no=(\d+)', chapter_url)
        chapter_num = chapter_num.group(1) if chapter_num else "unknown"
        
        print(f"다운로드 진행: {webtoon_title} - {chapter_num}화")
        
        # 이미지 요소 찾기
        image_tags = browser.find_elements(By.CSS_SELECTOR, ".wt_viewer img")
        
        if not image_tags:
            print(f"이미지를 찾을 수 없음: {chapter_num}화")
            return False
        
        # 저장 폴더 생성
        save_dir = os.path.join('webtoon_images', webtoon_title, chapter_num)
        os.makedirs(save_dir, exist_ok=True)
        
        # 요청 헤더 설정
        request_headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Referer': chapter_url
        }
        
        # 이미지 다운로드
        for idx, img_tag in enumerate(image_tags, 1):
            image_src = img_tag.get_attribute('src')
            if not image_src:
                continue
            
            # URL 정규화
            if image_src.startswith('//'):
                image_src = 'https:' + image_src
            elif image_src.startswith('/'):
                image_src = 'https://comic.naver.com' + image_src
            
            try:
                # 이미지 요청
                img_data = requests.get(image_src, headers=request_headers)
                img_data.raise_for_status()
                
                # 파일 확장자 결정
                file_ext = os.path.splitext(urlparse(image_src).path)[1] or '.jpg'
                output_filename = f"{idx:03d}{file_ext}"
                output_path = os.path.join(save_dir, output_filename)
                
                # 파일 저장
                with open(output_path, 'wb') as img_file:
                    img_file.write(img_data.content)
                
                print(f"  → 저장 완료: {output_filename}")
                
            except Exception as err:
                print(f"  → 이미지 다운로드 오류: {image_src} - {err}")
        
        return True
        
    except Exception as err:
        print(f"챕터 다운로드 실패: {chapter_url} - {err}")
        return False
    finally:
        if browser:
            browser.quit()

def get_webtoon_chapters(webtoon_title, list_url):
    """
    웹툰의 모든 챕터를 다운로드하는 메인 함수
    
    Args:
        webtoon_title (str): 웹툰 제목
        list_url (str): 웹툰 챕터 목록 페이지 URL
    """
    browser = None
    try:
        print(f"'{webtoon_title}' 웹툰 챕터 목록 수집 중...")
        
        # 브라우저 초기화
        browser = initialize_webdriver()
        
        # 목록 페이지 접속
        browser.get(list_url)
        
        # 페이지 로딩 대기
        time.sleep(4)
        
        # 챕터 링크 수집
        chapter_urls = []
        
        # 다양한 방법으로 링크 수집 시도
        selectors = [
            "table.viewList td.title a",
            "a[href*='/webtoon/detail']",
            "a[href*='titleId=']"
        ]
        
        for selector in selectors:
            try:
                links = browser.find_elements(By.CSS_SELECTOR, selector)
                for link in links:
                    href = link.get_attribute('href')
                    if href and '/webtoon/detail' in href and 'titleId=823933' in href:
                        chapter_urls.append(href)
                if chapter_urls:
                    break
            except:
                continue
        
        # 중복 제거
        chapter_urls = list(dict.fromkeys(chapter_urls))
        
        if not chapter_urls:
            print("챕터 링크를 찾을 수 없음")
            return
        
        print(f"총 {len(chapter_urls)}개의 챕터 발견")
        
        # 각 챕터 처리
        downloaded = 0
        for current, url in enumerate(chapter_urls, 1):
            print(f"\n[{current}/{len(chapter_urls)}] 처리 중...")
            
            if fetch_webtoon_images(webtoon_title, url):
                downloaded += 1
            
            # 서버 부하 방지
            time.sleep(1.5)
        
        print(f"\n처리 완료: {downloaded}/{len(chapter_urls)}개 챕터 성공")
        
    except Exception as err:
        print(f"다운로드 실패: {err}")
    finally:
        if browser:
            browser.quit()

# 프로그램 실행
if __name__ == "__main__":
    # 필수 패키지 확인
    required_packages = ['selenium', 'requests', 'bs4']
    missing = []
    
    for package in required_packages:
        try:
            __import__(package)
        except ImportError:
            missing.append(package)
    
    if missing:
        print("다음 패키지가 설치되지 않았습니다:")
        print(", ".join(missing))
        print("설치 명령어: pip install " + " ".join(missing))
        exit()
    
    print("모든 필수 패키지가 설치되어 있습니다.")
    
    # 배달왕 웹툰 다운로드 실행
    get_webtoon_chapters('배달왕', 'https://comic.naver.com/webtoon/list?titleId=823933')